# ETL-Prozess
---

Dieses Modul ist für das Extrahieren, Transformieren und Laden von Daten aus verschiedenen Quellen zuständig. Dieses Modul erstellt das Datenmodell, das die Basisparameter für die Simulationen enthält.

In [11]:
# Zum Starten des ETL-Prozesses führen Sie diese Zelle aus.

def run_etl_processes():
    try:
        data_model = run_ETL_Stromboerse()
        if data_model is not None:
            output_folder = "SGS/Output-Data"
            output_file = "ETL-Prozess_Ergebnisse.xlsx"
            output_path = os.path.join(output_folder, output_file)
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)
            data_model.to_excel(output_path, index=False)
            return True
    except Exception as e:
        print(f"Ein Fehler ist beim Speichern der Daten in der Excel-Datei aufgetreten: {e}")

### ETL-Prozess: Standardlastprofil
---
Diese Komponente ist für das Extrahieren, Transformieren und Laden von Daten aus der Quelle [Stromnetz Hamburg](https://www.stromnetz-hamburg.de/fuer-partner/stromlieferanten/zaehlverfahren) zuständig. Die Komponente verarbeitet eine Quelldatei, die ein [synthetisches Standardlastprofil](https://www.enargus.de/pub/bscw.cgi/d12072-2/*/*/Standardlastprofil.html?op=Wiki.getwiki#:~:text=Das%20synthetische%20Lastprofilverfahren%20stellt%20eine,gewichtet%20und%20anschließend%20aufsummiert%20werden.) darstellt.

#### Funktionsweise
In diesem Prozess werden die folgenden Funktionen verwendet:

1. `extraction_and_processing_dataset_Stromnetz_Hamburg()`: Die Funktion extrahiert die Daten aus einer [Quelldatei](https://filehub.admiralcloud.com/v5/deliverFile/76eee0f0-6112-4ca5-8f39-96d53f5871b8?download=true) und berechnet die Basisprozentwerte, die als Referenzwerte in den Simulationen zu verwenden sind.


2. `run_ETL_Standardlastprofil()`: Funktion zur Ausführung des Prozesses.

#### Quellcode

In [12]:
import os
import pandas as pd
from datetime import datetime

def extraction_and_processing_dataset_Stromnetz_Hamburg():
    try:
        input_folder = "SGS/Datenquelle/Stromnetz Hamburg-Daten"
        input_file = "Standardlastprofil Haushalt 2023.xlsx"
        input_path = os.path.join(input_folder, input_file)
        
        # Öffnen der Excel-Datei und Auswahl der gewünschten Spalten
        results_df = pd.read_excel(input_path, usecols=["Datum", "Uhrzeit", "SLP-Haushalt HH0 [kWh]"], parse_dates={"Zeitstempel": ["Datum", "Uhrzeit"]})
        
        # Berechnung des Prozentsatzes
        gesamtverbrauch_kWh = results_df['SLP-Haushalt HH0 [kWh]'].sum()
        results_df['SLP-Basisprozentwert'] = (results_df['SLP-Haushalt HH0 [kWh]'] / gesamtverbrauch_kWh) * 100

        print("- ETL-Prozess: Aufgabe 1/3 abgeschlossen")
        
        return results_df
    
    except Exception as e:
        print(f"Ein Fehler ist aufgetreten: {e}")
        return None

def run_ETL_Standardlastprofil():
    return extraction_and_processing_dataset_Stromnetz_Hamburg()

### ETL-Prozess: PV-Ertrag
---
Diese Komponente ist für das Extrahieren, Transformieren und Laden von Daten aus der Quellen [Deutscher Wetterdienst](https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/solar/BESCHREIBUNG_obsgermany_climate_hourly_solar_de.pdf) und [Photovoltaic Geographical Information System](https://joint-research-centre.ec.europa.eu/photovoltaic-geographical-information-system-pvgis_en) zuständig. Die Komponente verarbeitet Quelldateien mit Globalstrahlungsdaten und simulierten PV-Ertragswerten.
#### Funktionsweise

In diesem Prozess werden die folgenden Funktionen verwendet:

1. `extraction_dataset_dwd()`: Diese Funktion extrahiert die Daten aus einer [Quelldatei](https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/solar/stundenwerte_ST_01975_row.zip), die den stündlichen Globalstrahlungsverlauf in Hamburg von 2005 bis heute enthält.


2. `processing_dataset_dwd()`: Die Funktion verarbeitet die Globalstrahlungsdaten und gibt sie in 15-Minuten-Intervallen weiter.


3. `extraction_dataset_pvgis()`: Diese Funktion extrahiert Daten aus einer Quelldatei, die im [PVGIS-Simulator](https://re.jrc.ec.europa.eu/pvg_tools/de/#SA) definierte Parameter enthält. Diese Werte werden verwendet, um die Stromerzeugung von PV-Anlagen in 15-Minuten-Intervallen zu berechnen.


4. `processing_dataset_pvgis()`: Diese Funktion berechnet die Stromerzeugung der PV-Anlagen in 15-Minuten-Intervallen. Die Berechnungen werden für PV-Anlagen mit einer Gesamtleistung von 1 kWp skaliert.


5. `run_ETL_PV_Ertrag()`: Funktion zur Ausführung des Prozesses.

#### Quellcode

In [13]:
def extraction_dataset_dwd():
    try:
        # Erhaltung des aktuellen Arbeitsverzeichnisses
        cwd = os.getcwd()
        input_folder = "SGS/Datenquelle/DWD-Daten/Stundenwerte_Globalstrahlung"
        # Festlegung des Verzeichnisses, in dem sich die Datei befindet
        data_directory = os.path.join(cwd, input_folder)

        # Angeben des Dateinamens
        input_file = "produkt_st_stunde_20050101_20240229_01975.txt"
        # Kombinieren von Verzeichnis und Dateiname
        input_path = os.path.join(data_directory, input_file)

        # Überprüfen, ob die Datei vorhanden ist
        if not os.path.exists(input_path):
            raise FileNotFoundError(f"Fehler: Datei '{input_file}' nicht im Verzeichnis '{data_directory}' gefunden.")

        # Lesen der Daten aus der Textdatei und Interpretation von 'MESS_DATUM' als Datum
        data = pd.read_csv(input_path, sep=";", usecols=["MESS_DATUM", "FG_LBERG"], parse_dates=["MESS_DATUM"])

        # Filtern der Daten nach dem Zeitbereich im String-Format
        start_date_str = "2023010101:23"
        end_date_str = "2024010100:23"
        filtered_data = data[(data["MESS_DATUM"] >= start_date_str) & (data["MESS_DATUM"] <= end_date_str)]

        return filtered_data

    except FileNotFoundError as e:
        print(f"Fehler beim Lesen der Datei: {e}")

    except Exception as e:
        print(f"Fehler beim Verarbeiten der Daten: {e}")

def extraction_dataset_pvgis():
    try:
        # Angeben von Dateinamen und Blattnamen
        input_folder = "SGS/Datenquelle/PVGIS-Daten/Solardatenparameter_2023"
        input_file = "PVGIS-Daten_AVG-PV-Ertrag 2023.xlsx"
        input_sheet_name = "PVGIS-Daten_AVG-PV-Ertrag 2023"

        # Kombinieren der Verzeichnisse
        input_path = os.path.join(input_folder, input_file)

        # Überprüfen, ob die Eingabedatei vorhanden ist
        if not os.path.exists(input_path):
            raise FileNotFoundError(f"Fehler: Datei '{input_file}' nicht im Verzeichnis '{input_folder}' gefunden.")

        # Lesen der Daten aus der Quelldatei
        pvgis_data = pd.read_excel(input_path, sheet_name=input_sheet_name)

        return pvgis_data

    except FileNotFoundError as e:
        print(f"Fehler beim Lesen der Datei: {e}")

    except Exception as e:
        print(f"Fehler beim Verarbeiten der Daten: {e}")

def processing_dataset_dwd(filtered_data):
    try:
        # Extrahieren der Werte von "FG_LBERG"
        fg_lberg_values = filtered_data["FG_LBERG"].tolist()

        # Überprüfen, ob der Wert kleiner als 0 ist, falls ja, dann auf den Wert der vorherigen Zelle setzen
        for i in range(len(fg_lberg_values)):
            if fg_lberg_values[i] is not None and fg_lberg_values[i] < 0:
                fg_lberg_values[i] = fg_lberg_values[i-1] if i > 0 else None

        # Erstellen einer neuen Spalte "Globalstrahlung" basierend auf den bereinigten Werten von "FG_LBERG"
        global_radiation_values = []
        for fg_lberg_value in fg_lberg_values:
            if fg_lberg_value is not None:
                # Teilen des Werts von "FG_LBERG" durch 4 und Hinzufügen der Ergebnisse zur Liste
                global_radiation_values.extend([fg_lberg_value / 4] * 4)
            else:
                # Falls der Wert von "FG_LBERG" None ist, Hinzufügen von 4 leeren Zellen
                global_radiation_values.extend([None] * 4)

        results_df = run_ETL_Standardlastprofil()
        results_df['Globalstrahlung'] = global_radiation_values

        return results_df

    except Exception as e:
        print(f"Ein Fehler ist aufgetreten: {e}")

def processing_dataset_pvgis(results_df, pvgis_data):
    try:       
        # Variabel zur Speicherung der monatlichen Summen
        monthly_totals = {}

        # Iteration über die Zeilen des DataFrames, um die monatliche Summe der Globalstrahlung zu berechnen
        for index, row in results_df.iterrows():
            datum = row['Zeitstempel']
            globalstrahlung = row['Globalstrahlung']

            # Überprüfen, ob das Datum gültig ist
            if datum is not None:
                # Extrahieren des Monats aus dem Datum
                month = datum.strftime('%B')

                # Speicherung der monatlichen Summen
                if month not in monthly_totals:
                    monthly_totals[month] = {'sum_fg_lberg': 0}

                # Überprüfen, ob die Globalstrahlung gültig ist
                if globalstrahlung is not None:
                    # Kumulative Summe der Globalstrahlung für jeden Monat
                    monthly_totals[month]['sum_fg_lberg'] += globalstrahlung

        # Variabel zur Speicherung der monatlichen PVGIS-Leistungsdaten
        total_month_kwp = {}
        for row in pvgis_data.iterrows():
            month_performance = row[1]['Month']
            total_10kwp = row[1]['Total Month_10 kWp']
            total_month_kwp[month_performance] = {'10kWp': total_10kwp}

        # Iteration über die Monate und Berechnung der Werte "10 kWp"
        for month, data in monthly_totals.items():
            sum_globalstrahlung = data['sum_fg_lberg']
            # Überprüfen, ob der Monat in "total_month_kwp" vorhanden ist
            if month in total_month_kwp:
                gesamtmonat_kwp = total_month_kwp[month]
                # Berechnung und Einfügen der Werte "1 kWp"
                for index, row in results_df.iterrows():
                    datum = row['Zeitstempel']
                    globalstrahlung = row['Globalstrahlung']
                    if isinstance(datum, str):
                        datum = datetime.strptime(datum, '%Y-%m-%d')
                    if datum is not None and datum.strftime('%B') == month:
                        ten_kwp = (globalstrahlung * gesamtmonat_kwp['10kWp']) / sum_globalstrahlung
                        results_df.at[index, 'PV-1kWp'] = ten_kwp / 10
            else:
                print(f"Warnung: Keine Daten für den Monat {month} in 'PVGIS-Data' gefunden.")
                
        print("- ETL-Prozess: Aufgabe 2/3 abgeschlossen")
        
        return results_df

    except Exception as e:
        print(f"Ein Fehler ist aufgetreten: {e}")

def run_ETL_PV_Ertrag():        
          
    return processing_dataset_pvgis(processing_dataset_dwd(extraction_dataset_dwd()), extraction_dataset_pvgis())


### ETL-Prozess: Strombörse
---
Diese Komponente ist für das Extrahieren, Transformieren und Laden von Daten aus der Quelle [SMARD](https://www.smard.de/home/downloadcenter/download-marktdaten/?downloadAttributes=%7B%22selectedCategory%22:3,%22selectedSubCategory%22:8,%22selectedRegion%22:%22DE%22,%22selectedFileType%22:%22XLSX%22,%22from%22:1672527600000,%22to%22:1704063599999%7D) zuständig. Die Komponente verarbeitet eine Quelldatei, die die auf dem Day-Ahead-Markt ([EPEX SPOT](https://www.epexspot.com/en/market-data?market_area=DE-LU&trading_date=2024-03-31&delivery_date=2024-04-01&underlying_year=&modality=Auction&sub_modality=DayAhead&technology=&product=60&data_mode=graph&period=&production_period=)) festgelegten Strompreise darstellt.
#### Funktionsweise
In diesem Prozess werden die folgenden Funktionen verwendet:



1. `extraction_and_processing_dataset_SMARD()`: Diese Funktion extrahiert aus einer Quelldatei die Marktdaten zu den Großhandelsstrompreisen für das Jahr 2023. Diese Daten stellen die Strompreise dar, die auf dem Day-Ahead-Markt in 15-Minuten-Intervallen festgelegt werden.


2. `run_ETL_Stromboerse()`: Funktion zur Ausführung des Prozesses.

#### Quellcode

In [14]:
def extraction_and_processing_dataset_SMARD():
    try:
        input_folder = "SGS/Datenquelle/SMARD-Daten/Viertelstunde"
        input_file = "Gro_handelspreise_202301010000_202312312359_Viertelstunde.xlsx"
        input_path = os.path.join(input_folder, input_file)
        
        results_df = run_ETL_PV_Ertrag()
        
        # Lesen der Daten aus der Excel-Datei und Laden in einen DataFrame
        df = pd.read_excel(input_path, sheet_name="Großhandelspreise", header=9, usecols="D")
        
        # Umbenennen der Spalte
        df.columns = ['Strommarktpreis [€/MWh]']

        # Umrechnen des Preises in €/kWh
        df['Strommarktpreis [€/kWh]'] = df['Strommarktpreis [€/MWh]'] / 1000
        
        # Hinzufügen der neuen Spalten zum bestehenden DataFrame results_df
        results_df['Strommarktpreis [€/MWh]'] = df['Strommarktpreis [€/MWh]']
        results_df['Strommarktpreis [€/kWh]'] = df['Strommarktpreis [€/kWh]']

        print("- ETL-Prozess: Aufgabe 3/3 abgeschlossen")
        
        return results_df

    except Exception as e:
        print("Ein Fehler ist aufgetreten:", e)

def run_ETL_Stromboerse():
    return extraction_and_processing_dataset_SMARD()
